In [45]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer, AerSimulator
from qiskit.quantum_info import Statevector, state_fidelity
from qiskit_aer.noise import NoiseModel, amplitude_damping_error, depolarizing_error, phase_damping_error, pauli_error, kraus_error
import time
import tracemalloc
import csv


In [66]:
# Initialize the AerSimulator for statevector simulation
backend = AerSimulator(method='statevector')

In [67]:
# Define Quantum Error Correction Code Circuits
def shor_code_circuit(include_measure=True):
    qc = QuantumCircuit(9)
    
    # Encode logical |0> state (bit-flip encoding)
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(0, 2)
    
    qc.cx(3, 4)
    qc.cx(3, 5)
    
    qc.cx(6, 7)
    qc.cx(6, 8)
    
    # Phase-flip encoding (using CNOTs and H gates)
    for i in [0, 3, 6]:
        qc.h(i)
        qc.cx(i, i+1)
        qc.cx(i, i+2)
        qc.h(i)
    
    # Bit-flip protection
    qc.cx(0, 3)
    qc.cx(0, 6)
    qc.cx(1, 4)
    qc.cx(1, 7)
    qc.cx(2, 5)
    qc.cx(2, 8)

    # Save statevector
    qc.save_statevector()

    # Optionally include measurement
    if include_measure:
        qc.measure_all()
    
    return qc

def steane_code_circuit(include_measure=True):
    qc = QuantumCircuit(7)

    # Encode logical |0> state
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(0, 2)
    
    qc.cx(0, 3)
    qc.cx(1, 4)
    qc.cx(2, 5)
    
    qc.h(0)
    qc.h(1)
    qc.h(2)
    
    qc.cx(0, 6)

    # Save statevector
    qc.save_statevector()

    # Optionally include measurement
    if include_measure:
        qc.measure_all()
    
    return qc

def surface_code_circuit(include_measure=True):
    qc = QuantumCircuit(13)

    # Simplified example of a surface code (not a full implementation)
    # Initialize logical |0> state
    qc.h([0, 1, 2])
    
    # Create stabilizers
    for i in range(1, 7):
        qc.cx(0, i)
        qc.cz(0, i+6)

    # Save statevector
    qc.save_statevector()

    # Optionally include measurement
    if include_measure:
        qc.measure_all()
    
    return qc

In [68]:



# Noise Models
def create_depolarizing_noise_model(noise_level):
    noise_model = NoiseModel()
    depol_error_1q = depolarizing_error(noise_level, 1)
    depol_error_2q = depolarizing_error(noise_level, 2)
    noise_model.add_all_qubit_quantum_error(depol_error_1q, ["u3", "u2", "u1"])
    noise_model.add_all_qubit_quantum_error(depol_error_2q, ["cx"])
    return noise_model

def create_amplitude_damping_noise_model(noise_level):
    noise_model = NoiseModel()
    amp_damp_error_1q = amplitude_damping_error(noise_level)
    noise_model.add_all_qubit_quantum_error(amp_damp_error_1q, ["u3"])
    return noise_model

def create_phase_damping_noise_model(noise_level):
    noise_model = NoiseModel()
    phase_damp_error_1q = phase_damping_error(noise_level)
    noise_model.add_all_qubit_quantum_error(phase_damp_error_1q, ["u1"])
    return noise_model

def create_bit_flip_noise_model(noise_level):
    noise_model = NoiseModel()
    bit_flip_error = pauli_error([('X', noise_level), ('I', 1 - noise_level)])
    noise_model.add_all_qubit_quantum_error(bit_flip_error, ["x"])
    return noise_model

def create_phase_flip_noise_model(noise_level):
    noise_model = NoiseModel()
    phase_flip_error = pauli_error([('Z', noise_level), ('I', 1 - noise_level)])
    noise_model.add_all_qubit_quantum_error(phase_flip_error, ["z"])
    return noise_model

def create_bit_phase_flip_noise_model(noise_level):
    noise_model = NoiseModel()
    bit_phase_flip_error = pauli_error([('Y', noise_level), ('I', 1 - noise_level)])
    noise_model.add_all_qubit_quantum_error(bit_phase_flip_error, ["y"])
    return noise_model

def create_kraus_error_model(noise_level):
    noise_model = NoiseModel()
    K0 = np.array([[1, 0], [0, np.sqrt(1 - noise_level)]])
    K1 = np.array([[0, np.sqrt(noise_level)], [0, 0]])
    kraus_ops = [K0, K1]
    error = kraus_error(kraus_ops)
    noise_model.add_all_qubit_quantum_error(error, ["h", "u3", "x", "z"])
    return noise_model

In [69]:

# Validate Circuit without noise
def validate_circuit_ideal(code_name):
    if code_name == 'shor':
        qc_circuit = shor_code_circuit()
    elif code_name == 'steane':
        qc_circuit = steane_code_circuit()
    elif code_name == 'surface':
        qc_circuit = surface_code_circuit()
    else:
        raise ValueError(f"Unknown code name: {code_name}")
    
    # Transpile the circuit for the backend
    qc_transpiled = transpile(qc_circuit, backend)
    
    # Run the circuit and retrieve the statevector
    try:
        result = backend.run(qc_transpiled).result()
        statevector = result.get_statevector(qc_transpiled)
        print(f"Statevector for {code_name} code:\n", statevector)
    except Exception as e:
        print(f"Error in retrieving statevector for {code_name} code: {str(e)}")

# Validate each QEC circuit without noise
validate_circuit_ideal('shor')
validate_circuit_ideal('steane')
validate_circuit_ideal('surface')

Statevector for shor code:
 Statevector([ 2.77555756e-17+2.16489014e-17j,
              1.76776695e-01-2.16489014e-17j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              1.76776695e-01-6.49467042e-17j,
             -2.77555756e-17+1.14203872e-32j,
              2.77555756e-17+2.16489014e-17j,
              1.76776695e-01-2.16489014e-17j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
             -1.76776695e-01+6.49467042e-17j,
              2.77555756e-17-1.14203872e-32j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
      

In [70]:

def calculate_error_rate(ideal_state, noisy_state):
    # Calculate the probability that the state differs from the ideal state
    num_errors = sum(1 for i in range(len(ideal_state)) if ideal_state[i] != noisy_state[i])
    error_rate = num_errors / len(ideal_state)
    return error_rate

def simulate_with_qec(noise_model, backend, code_name):
    if code_name == 'shor':
        qc_circuit = shor_code_circuit()
    elif code_name == 'steane':
        qc_circuit = steane_code_circuit()
    elif code_name == 'surface':
        qc_circuit = surface_code_circuit()
    else:
        raise ValueError(f"Unknown code name: {code_name}")
    
    # Transpile the circuit for the backend
    qc_transpiled = transpile(qc_circuit, backend)
    
    # Run the simulation with noise to get the statevector
    try:
        result = backend.run(qc_transpiled, noise_model=noise_model).result()
        state_noisy = result.get_statevector(qc_transpiled)
        
        # Calculate fidelity with the ideal state
        num_qubits_transpiled = qc_transpiled.num_qubits
        state_ideal = Statevector.from_label('0' * num_qubits_transpiled)
        fidelity = state_fidelity(state_noisy, state_ideal)

        # Calculate error rate
        error_rate = calculate_error_rate(state_ideal, state_noisy)
        
        return fidelity, error_rate
    except Exception as e:
        print(f"Error in simulation for {code_name} code: {str(e)}")
        return None, None

# Simulate with different noise models and QEC codes
def simulate_with_all_noise_models(noise_level, backend):
    noise_models = {
        "Depolarizing": create_depolarizing_noise_model(noise_level),
        "Amplitude Damping": create_amplitude_damping_noise_model(noise_level),
        "Phase Damping": create_phase_damping_noise_model(noise_level),
        "Bit Flip": create_bit_flip_noise_model(noise_level),
        "Phase Flip": create_phase_flip_noise_model(noise_level),
        "Bit Phase Flip": create_bit_phase_flip_noise_model(noise_level),
        "Kraus": create_kraus_error_model(noise_level)
    }
    
    codes = ["shor", "steane", "surface"]
    results = []
    
    for code_name in codes:
        for noise_name, noise_model in noise_models.items():
            fidelity, error_rate = simulate_with_qec(noise_model, backend, code_name)
            results.append({
                "Code": code_name,
                "Noise Model": noise_name,
                "Fidelity": fidelity,
                "Error Rate": error_rate
            })
            print(f"Code: {code_name}, Noise Model: {noise_name}, Fidelity: {fidelity}, Error Rate: {error_rate}")
    
    return results

# Example: Simulate all codes with all noise models
noise_level = 0.01  # You can adjust this level as needed
results = simulate_with_all_noise_models(noise_level, backend)


Code: shor, Noise Model: Depolarizing, Fidelity: 1.2390469098367252e-33, Error Rate: 0.125
Code: shor, Noise Model: Amplitude Damping, Fidelity: 1.2390469098367252e-33, Error Rate: 0.125
Code: shor, Noise Model: Phase Damping, Fidelity: 1.2390469098367252e-33, Error Rate: 0.125
Code: shor, Noise Model: Bit Flip, Fidelity: 1.2390469098367252e-33, Error Rate: 0.125
Code: shor, Noise Model: Phase Flip, Fidelity: 1.2390469098367252e-33, Error Rate: 0.125
Code: shor, Noise Model: Bit Phase Flip, Fidelity: 1.2390469098367252e-33, Error Rate: 0.125
Code: shor, Noise Model: Kraus, Fidelity: 8.091489318035219e-07, Error Rate: 0.125
Code: steane, Noise Model: Depolarizing, Fidelity: 0.06250000000000003, Error Rate: 0.125
Code: steane, Noise Model: Amplitude Damping, Fidelity: 0.06250000000000003, Error Rate: 0.125
Code: steane, Noise Model: Phase Damping, Fidelity: 0.06250000000000003, Error Rate: 0.125
Code: steane, Noise Model: Bit Flip, Fidelity: 0.06250000000000003, Error Rate: 0.125
Code: s

In [73]:
def simulate_no_correction(noise_model, backend):
    # Simple circuit without QEC
    qc = QuantumCircuit(1, 1)
    qc.h(0)
    
    # Transpile for the backend (without measurement for statevector)
    qc_transpiled_no_measure = transpile(qc, backend)
    
    # Evolve state vector without measurement
    state_noisy = Statevector.from_int(0, 2)
    state_noisy = state_noisy.evolve(qc_transpiled_no_measure)
    fidelity = state_fidelity(state_noisy, Statevector([1, 0]))  # Ideal state
    
    # Add measurement and transpile again for getting counts
    qc.measure(0, 0)
    qc_transpiled_with_measure = transpile(qc, backend)
    
    # Run the simulation with noise to get the error rate
    result = backend.run(qc_transpiled_with_measure, noise_model=noise_model, shots=1024).result()
    counts = result.get_counts()
    error_rate = 1 - (counts.get('0', 0) / 1024)
    
    return error_rate, fidelity

# Example: Simulate error rates and fidelity before applying QEC codes
noise_level = 0.05  # Adjust noise level as needed
results = simulate_with_all_noise_models(noise_level, backend)

#print(f"Error Rate without QEC: {error_rate_no_correction}")
#print(f"Fidelity without QEC: {fidelity_no_correction}")


Code: shor, Noise Model: Depolarizing, Fidelity: 0.0, Error Rate: 0.095703125
Code: shor, Noise Model: Amplitude Damping, Fidelity: 1.2390469098367252e-33, Error Rate: 0.125
Code: shor, Noise Model: Phase Damping, Fidelity: 1.2390469098367252e-33, Error Rate: 0.125
Code: shor, Noise Model: Bit Flip, Fidelity: 1.2390469098367252e-33, Error Rate: 0.125
Code: shor, Noise Model: Phase Flip, Fidelity: 1.2390469098367252e-33, Error Rate: 0.125
Code: shor, Noise Model: Bit Phase Flip, Fidelity: 1.2390469098367252e-33, Error Rate: 0.125
Code: shor, Noise Model: Kraus, Fidelity: 2.331834530286036e-05, Error Rate: 0.125
Code: steane, Noise Model: Depolarizing, Fidelity: 0.06250000000000003, Error Rate: 0.125
Code: steane, Noise Model: Amplitude Damping, Fidelity: 0.06250000000000003, Error Rate: 0.125
Code: steane, Noise Model: Phase Damping, Fidelity: 0.06250000000000003, Error Rate: 0.125
Code: steane, Noise Model: Bit Flip, Fidelity: 0.06250000000000003, Error Rate: 0.125
Code: steane, Noise 

In [78]:
def test_with_gradual_noise(backend, code_name, initial_noise_level=0.001, step=0.001, max_noise_level=0.01):
    noise_level = initial_noise_level
    while noise_level <= max_noise_level:
        noise_models = {
            "Depolarizing": create_depolarizing_noise_model(noise_level),
            "Amplitude Damping": create_amplitude_damping_noise_model(noise_level),
            "Phase Damping": create_phase_damping_noise_model(noise_level),
            "Bit Flip": create_bit_flip_noise_model(noise_level),
            "Phase Flip": create_phase_flip_noise_model(noise_level),
            "Bit Phase Flip": create_bit_phase_flip_noise_model(noise_level),
            "Kraus": create_kraus_error_model(noise_level)
        }
        
        for noise_name, noise_model in noise_models.items():
            try:
                fidelity = simulate_with_qec(noise_model, backend, code_name)
                print(f"Noise Level: {noise_level}, Code: {code_name}, Noise Model: {noise_name}, Fidelity: {fidelity}")
            except Exception as e:
                print(f"Error in simulation for {code_name} with {noise_name}: {str(e)}")

        noise_level += step

# Gradually introduce noise for Shor code
test_with_gradual_noise(backend, 'shor')
test_with_gradual_noise(backend, 'steane')
test_with_gradual_noise(backend, 'surface')


Noise Level: 0.001, Code: shor, Noise Model: Depolarizing, Fidelity: (1.2390469098367252e-33, 0.125)
Noise Level: 0.001, Code: shor, Noise Model: Amplitude Damping, Fidelity: (1.2390469098367252e-33, 0.125)
Noise Level: 0.001, Code: shor, Noise Model: Phase Damping, Fidelity: (1.2390469098367252e-33, 0.125)
Noise Level: 0.001, Code: shor, Noise Model: Bit Flip, Fidelity: (1.2390469098367252e-33, 0.125)
Noise Level: 0.001, Code: shor, Noise Model: Phase Flip, Fidelity: (1.2390469098367252e-33, 0.125)
Noise Level: 0.001, Code: shor, Noise Model: Bit Phase Flip, Fidelity: (1.2390469098367252e-33, 0.125)
Noise Level: 0.001, Code: shor, Noise Model: Kraus, Fidelity: (7.839898519637638e-09, 0.125)
Noise Level: 0.002, Code: shor, Noise Model: Depolarizing, Fidelity: (1.2390469098367252e-33, 0.125)
Noise Level: 0.002, Code: shor, Noise Model: Amplitude Damping, Fidelity: (1.2390469098367252e-33, 0.125)
Noise Level: 0.002, Code: shor, Noise Model: Phase Damping, Fidelity: (1.2390469098367252e-3

In [80]:
def analyze_circuit_depth(code_name):
    if code_name == 'shor':
        qc_circuit = shor_code_circuit(include_measure=False)
    elif code_name == 'steane':
        qc_circuit = steane_code_circuit(include_measure=False)
    elif code_name == 'surface':
        qc_circuit = surface_code_circuit(include_measure=False)
    else:
        raise ValueError(f"Unknown code name: {code_name}")
    
    # Transpile and analyze depth
    qc_transpiled = transpile(qc_circuit, backend)
    depth = qc_transpiled.depth()
    print(f"Circuit depth for {code_name} code: {depth}")

# Analyze circuit depth
analyze_circuit_depth('shor')
analyze_circuit_depth('steane')
analyze_circuit_depth('surface')


Circuit depth for shor code: 9
Circuit depth for steane code: 6
Circuit depth for surface code: 13


In [107]:


def simulate_code_threshold(noise_levels, code_name):
    thresholds = []
    fidelities = []
    results = []

    backend = AerSimulator()

    print(f"Starting simulation for {code_name} code.")

    for noise_level in noise_levels:
        print(f"Simulating for noise level: {noise_level}")

        noise_models = {
            "Depolarizing": create_depolarizing_noise_model(noise_level),
            "Amplitude Damping": create_amplitude_damping_noise_model(noise_level),
            "Phase Damping": create_phase_damping_noise_model(noise_level),
            "Bit Flip": create_bit_flip_noise_model(noise_level),
            "Phase Flip": create_phase_flip_noise_model(noise_level),
            "Bit Phase Flip": create_bit_phase_flip_noise_model(noise_level),
            "Kraus": create_kraus_error_model(noise_level)
        }

        for noise_name, noise_model in noise_models.items():
            print(f"Running simulation for noise model: {noise_name}")
            try:
                # Select the code to simulate
                if code_name == 'shor':
                    qc_circuit = shor_code_circuit(include_measure=True)
                    ideal_output = Statevector.from_label('0' * 9)
                elif code_name == 'steane':
                    qc_circuit = steane_code_circuit(include_measure=True)
                    ideal_output = Statevector.from_label('0' * 7)
                elif code_name == 'surface':
                    qc_circuit = surface_code_circuit(include_measure=True)
                    ideal_output = Statevector.from_label('0' * 17)  # Adjust as per surface code
                else:
                    raise ValueError(f"Unknown code name: {code_name}")

                # Transpile the circuit for the backend
                qc_transpiled = transpile(qc_circuit, backend)
                print(f"Transpilation complete for {noise_name} at noise level {noise_level}.")

                # Run the simulation with the noise model
                job_noisy = backend.run(qc_transpiled, noise_model=noise_model, shots=1024)
                result_noisy = job_noisy.result(timeout=600)  # Set a longer timeout

                # Check if the result is available
                if not result_noisy.success:
                    raise RuntimeError(f"Simulation failed for {code_name} with {noise_name} at noise level {noise_level}")

                # Get counts and calculate the error rate
                counts_noisy = result_noisy.get_counts(qc_transpiled)
                if counts_noisy is None:
                    raise ValueError(f"No counts data found for circuit in {noise_name} at noise level {noise_level}")

                error_rate = 1 - (counts_noisy.get('0' * qc_circuit.num_clbits, 0) / 1024)

                # Calculate fidelity
                final_statevector = Statevector(result_noisy.get_statevector(qc_transpiled))
                fidelity = state_fidelity(final_statevector, ideal_output)

                # Store results
                thresholds.append(error_rate)
                fidelities.append(fidelity)
                results.append({
                    'Code': code_name,
                    'Noise_Model': noise_name,
                    'Noise_Level': noise_level,
                    'Error_Rate': error_rate,
                    'Fidelity': fidelity
                })

                print(f"Results for {code_name}, Noise Model: {noise_name}, Noise Level: {noise_level} - Error Rate: {error_rate}, Fidelity: {fidelity}")

                # Delay between simulations to avoid backend issues
                time.sleep(10)

            except Exception as e:
                print(f"Error in simulation for {code_name} with {noise_name}: {str(e)}")
                continue

    print(f"Simulation for {code_name} complete.")
    return thresholds, fidelities, results

# Example usage:
noise_levels = [0.01, 0.02, 0.03]  # Add or remove noise levels as needed
shor_thresholds, shor_fidelities, shor_results = simulate_code_threshold(noise_levels, 'shor')
steane_thresholds, steane_fidelities, steane_results = simulate_code_threshold(noise_levels, 'steane')
surface_thresholds, surface_fidelities, surface_results = simulate_code_threshold(noise_levels, 'surface')


Starting simulation for shor code.
Simulating for noise level: 0.01
Running simulation for noise model: Depolarizing
Transpilation complete for Depolarizing at noise level 0.01.
Results for shor, Noise Model: Depolarizing, Noise Level: 0.01 - Error Rate: 1.0, Fidelity: 1.2390469098367252e-33
Running simulation for noise model: Amplitude Damping
Transpilation complete for Amplitude Damping at noise level 0.01.
Results for shor, Noise Model: Amplitude Damping, Noise Level: 0.01 - Error Rate: 1.0, Fidelity: 1.2390469098367252e-33
Running simulation for noise model: Phase Damping
Transpilation complete for Phase Damping at noise level 0.01.
Results for shor, Noise Model: Phase Damping, Noise Level: 0.01 - Error Rate: 1.0, Fidelity: 1.2390469098367252e-33
Running simulation for noise model: Bit Flip
Transpilation complete for Bit Flip at noise level 0.01.
Results for shor, Noise Model: Bit Flip, Noise Level: 0.01 - Error Rate: 1.0, Fidelity: 1.2390469098367252e-33
Running simulation for noi

In [111]:
# Print the shapes and contents of the arrays
print("shor_thresholds shape:", shor_thresholds.shape, "content:", shor_thresholds)
print("steane_thresholds shape:", steane_thresholds.shape, "content:", steane_thresholds)
print("surface_thresholds shape:", surface_thresholds.shape, "content:", surface_thresholds)

print("shor_fidelities shape:", shor_fidelities.shape, "content:", shor_fidelities)
print("steane_fidelities shape:", steane_fidelities.shape, "content:", steane_fidelities)
print("surface_fidelities shape:", surface_fidelities.shape, "content:", surface_fidelities)


shor_thresholds shape: (21,) content: [1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         0.99804688 0.99804688 1.         1.         1.
 1.         1.         0.99902344]
steane_thresholds shape: (21,) content: [0.94433594 0.92871094 0.93066406 0.94042969 0.94921875 0.94433594
 0.92285156 0.953125   0.92871094 0.91992188 0.93945312 0.93847656
 0.93457031 0.93066406 0.93261719 0.94238281 0.93066406 0.94824219
 0.9375     0.92675781 0.9453125 ]
surface_thresholds shape: (0,) content: []
shor_fidelities shape: (21,) content: [1.23904691e-33 1.23904691e-33 1.23904691e-33 1.23904691e-33
 1.23904691e-33 1.23904691e-33 8.09148932e-07 0.00000000e+00
 1.23904691e-33 1.23904691e-33 1.23904691e-33 1.23904691e-33
 1.23904691e-33 3.35276921e-06 0.00000000e+00 1.23904691e-33
 1.23904691e-33 1.23904691e-33 1.23904691e-33 1.23904691e-33
 7.81589878e-06]
steane_fidelities shape: (21,) content: [0.0625     0.0625     0.0625  

In [ ]:

print("Surface code simulation result:", surface_result)
surface_thresholds.append(surface_error_rate)
surface_fidelities.append(surface_fidelity)


In [113]:
import numpy as np
import matplotlib.pyplot as plt

# Convert lists to numpy arrays
shor_thresholds = np.array(shor_thresholds)
steane_thresholds = np.array(steane_thresholds)
surface_thresholds = np.array(surface_thresholds)

shor_fidelities = np.array(shor_fidelities)
steane_fidelities = np.array(steane_fidelities)
surface_fidelities = np.array(surface_fidelities)

# Check the dimensions and compute means only if valid
if shor_thresholds.ndim == 2 and shor_thresholds.shape[1] == len(noise_levels):
    shor_mean_thresholds = np.mean(shor_thresholds, axis=0)
else:
    print("Issue with shor_thresholds dimensions")
    shor_mean_thresholds = np.array([])

if steane_thresholds.ndim == 2 and steane_thresholds.shape[1] == len(noise_levels):
    steane_mean_thresholds = np.mean(steane_thresholds, axis=0)
else:
    print("Issue with steane_thresholds dimensions")
    steane_mean_thresholds = np.array([])

if surface_thresholds.ndim == 2 and surface_thresholds.shape[1] == len(noise_levels):
    surface_mean_thresholds = np.mean(surface_thresholds, axis=0)
else:
    print("Issue with surface_thresholds dimensions")
    surface_mean_thresholds = np.array([])

if shor_fidelities.ndim == 2 and shor_fidelities.shape[1] == len(noise_levels):
    shor_mean_fidelities = np.mean(shor_fidelities, axis=0)
else:
    print("Issue with shor_fidelities dimensions")
    shor_mean_fidelities = np.array([])

if steane_fidelities.ndim == 2 and steane_fidelities.shape[1] == len(noise_levels):
    steane_mean_fidelities = np.mean(steane_fidelities, axis=0)
else:
    print("Issue with steane_fidelities dimensions")
    steane_mean_fidelities = np.array([])

if surface_fidelities.ndim == 2 and surface_fidelities.shape[1] == len(noise_levels):
    surface_mean_fidelities = np.mean(surface_fidelities, axis=0)
else:
    print("Issue with surface_fidelities dimensions")
    surface_mean_fidelities = np.array([])

# Ensure the arrays are not empty before plotting
if len(shor_mean_thresholds) > 0:
    plt.figure(figsize=(10, 6))
    plt.plot(noise_levels, shor_mean_thresholds, label="Shor Code Mean Error Rate", marker='o')
    plt.plot(noise_levels, steane_mean_thresholds, label="Steane Code Mean Error Rate", marker='o')
    plt.plot(noise_levels, surface_mean_thresholds, label="Surface Code Mean Error Rate", marker='o')
    plt.xlabel("Noise Probability")
    plt.ylabel("Mean Error Rate")
    plt.legend()
    plt.title("Mean Error Rate Comparison for Shor, Steane, and Surface Codes")
    plt.grid(True)
    plt.show()

if len(shor_mean_fidelities) > 0:
    plt.figure(figsize=(10, 6))
    plt.plot(noise_levels, shor_mean_fidelities, label="Shor Code Mean Fidelity", marker='o')
    plt.plot(noise_levels, steane_mean_fidelities, label="Steane Code Mean Fidelity", marker='o')
    plt.plot(noise_levels, surface_mean_fidelities, label="Surface Code Mean Fidelity", marker='o')
    plt.xlabel("Noise Probability")
    plt.ylabel("Mean Fidelity")
    plt.legend()
    plt.title("Mean Fidelity Comparison for Shor, Steane, and Surface Codes")
    plt.grid(True)
    plt.show()


Issue with shor_thresholds dimensions
Issue with steane_thresholds dimensions
Issue with surface_thresholds dimensions
Issue with shor_fidelities dimensions
Issue with steane_fidelities dimensions
Issue with surface_fidelities dimensions


In [114]:
def analyze_resources(backend, code_name):
    if code_name == 'shor':
        qc_circuit = shor_code_circuit(include_measure=True)
    elif code_name == 'steane':
        qc_circuit = steane_code_circuit(include_measure=True)
    elif code_name == 'surface':
        qc_circuit = surface_code_circuit(include_measure=True)
    else:
        raise ValueError(f"Unknown code name: {code_name}")
    
    # Transpile the circuit for the backend
    qc_transpiled = transpile(qc_circuit, backend)
    
    # Calculate resources
    depth = qc_transpiled.depth()
    gate_counts = qc_transpiled.count_ops()
    qubit_count = qc_transpiled.num_qubits
    
    # Calculate qubit overhead (physical qubits needed minus logical qubit)
    logical_qubit_count = 1  # Assuming we're encoding one logical qubit in these codes
    qubit_overhead = qubit_count - logical_qubit_count
    
    print(f"Code: {code_name}")
    print(f"Circuit Depth: {depth}")
    print(f"Gate Counts: {gate_counts}")
    print(f"Total Qubit Count: {qubit_count}")
    print(f"Qubit Overhead: {qubit_overhead}")
    
    return {
        'Code': code_name,
        'Circuit Depth': depth,
        'Gate Counts': gate_counts,
        'Total Qubit Count': qubit_count,
        'Qubit Overhead': qubit_overhead
    }

# Analyze resources and qubit overhead for each code
shor_resources = analyze_resources(backend, 'shor')
steane_resources = analyze_resources(backend, 'steane')
surface_resources = analyze_resources(backend, 'surface')

# Save resource usage results including qubit overhead
with open('code_resource_usage.csv', 'w', newline='') as csvfile:
    fieldnames = ['Code', 'Circuit Depth', 'Gate Counts', 'Total Qubit Count', 'Qubit Overhead']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    writer.writerow(shor_resources)
    writer.writerow(steane_resources)
    writer.writerow(surface_resources)


Code: shor
Circuit Depth: 10
Gate Counts: OrderedDict({'cx': 18, 'measure': 9, 'h': 7, 'save_statevector': 1, 'barrier': 1})
Total Qubit Count: 9
Qubit Overhead: 8
Code: steane
Circuit Depth: 7
Gate Counts: OrderedDict({'measure': 7, 'cx': 6, 'h': 4, 'save_statevector': 1, 'barrier': 1})
Total Qubit Count: 7
Qubit Overhead: 6
Code: surface
Circuit Depth: 14
Gate Counts: OrderedDict({'measure': 13, 'cx': 6, 'cz': 6, 'h': 3, 'save_statevector': 1, 'barrier': 1})
Total Qubit Count: 13
Qubit Overhead: 12


In [172]:
def simulate_fault_tolerance(noise_level, backend, codes, repetitions=10):
    noise_models = {
        'Depolarizing': create_depolarizing_noise_model(noise_level),
        'Amplitude Damping': create_amplitude_damping_noise_model(noise_level),
        'Phase Damping': create_phase_damping_noise_model(noise_level),
        'Bit Flip': create_bit_flip_noise_model(noise_level),
        'Phase Flip': create_phase_flip_noise_model(noise_level),
        'Bit Phase Flip': create_bit_phase_flip_noise_model(noise_level),
        'Kraus': create_kraus_error_model(noise_level)
    }
    
    all_results = {}
    
    for code_name in codes:
        results = []
        
        for noise_model_name, noise_model in noise_models.items():
            for i in range(repetitions):
                if code_name == 'shor':
                    qc_circuit = shor_code_circuit(include_measure=False)
                    ideal_output = Statevector.from_label('0' * 9)
                elif code_name == 'steane':
                    qc_circuit = steane_code_circuit(include_measure=False)
                    ideal_output = Statevector.from_label('0' * 7)
                elif code_name == 'surface':
                    qc_circuit = surface_code_circuit(include_measure=False)
                    ideal_output = Statevector.from_label('0' * 13)  # Adjusted qubit count
                else:
                    raise ValueError(f"Unknown code name: {code_name}")

                # Transpile the circuit with minimal optimization
                qc_transpiled = transpile(qc_circuit, backend, optimization_level=0)

                try:
                    # Get the ideal statevector by running on a noise-free simulator
                    ideal_backend = AerSimulator()
                    result_ideal = ideal_backend.run(qc_transpiled).result()
                    state_ideal = result_ideal.get_statevector(qc_transpiled)

                    # Run the noisy circuit
                    result_noisy = backend.run(qc_transpiled, noise_model=noise_model).result()
                    state_noisy = result_noisy.get_statevector(qc_transpiled)

                    # Calculate fidelity between the noisy and ideal statevectors
                    fidelity = state_fidelity(state_noisy, state_ideal)
                    
                    # Apply measurements to a copy of the transpiled circuit for error rate calculation
                    qc_transpiled_with_measure = qc_transpiled.copy()
                    qc_transpiled_with_measure.measure_all()
                    result_noisy_with_measurement = backend.run(qc_transpiled_with_measure, noise_model=noise_model, shots=1024).result()
                    counts_noisy = result_noisy_with_measurement.get_counts()

                    error_rate = 1 - (counts_noisy.get('0' * qc_transpiled_with_measure.num_clbits, 0) / 1024)
                    
                    results.append({
                        'Code': code_name,
                        'Noise Model': noise_model_name,
                        'Fidelity': fidelity,
                        'Error Rate': error_rate,
                        'Repetition': i + 1
                    })

                    print(f"Code: {code_name}, Noise Model: {noise_model_name}, Fidelity: {fidelity}, Error Rate: {error_rate}, Repetition: {i + 1}")

                except Exception as e:
                    print(f"Error in processing: {e}")
                    continue
        
        all_results[code_name] = results
    
    return all_results

# Usage
codes = ['shor', 'steane', 'surface']
backend = AerSimulator()
noise_level = 0.05
fault_tolerance_results = simulate_fault_tolerance(noise_level=noise_level, backend=backend, codes=codes, repetitions=10)

# Save results for each code to separate CSV files
for code_name, results in fault_tolerance_results.items():
    csv_filename = f'{code_name}_fault_tolerance_results.csv'
    with open(csv_filename, 'w', newline='') as csvfile:
        fieldnames = ['Repetition', 'Code', 'Noise Model', 'Error Rate', 'Fidelity']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for result in results:
            writer.writerow(result)

    print(f"Results for {code_name} saved to {csv_filename}")


Code: shor, Noise Model: Depolarizing, Fidelity: 0.0, Error Rate: 0.9990234375, Repetition: 1
Code: shor, Noise Model: Depolarizing, Fidelity: 0.0, Error Rate: 0.9970703125, Repetition: 2
Code: shor, Noise Model: Depolarizing, Fidelity: 1.0, Error Rate: 0.9990234375, Repetition: 3
Code: shor, Noise Model: Depolarizing, Fidelity: 4.930380657631324e-32, Error Rate: 0.99609375, Repetition: 4
Code: shor, Noise Model: Depolarizing, Fidelity: 2.1060252372395076e-32, Error Rate: 0.9990234375, Repetition: 5
Code: shor, Noise Model: Depolarizing, Fidelity: 0.0, Error Rate: 0.998046875, Repetition: 6
Code: shor, Noise Model: Depolarizing, Fidelity: 1.0, Error Rate: 1.0, Repetition: 7
Code: shor, Noise Model: Depolarizing, Fidelity: 1.0, Error Rate: 0.998046875, Repetition: 8
Code: shor, Noise Model: Depolarizing, Fidelity: 0.0, Error Rate: 0.998046875, Repetition: 9
Code: shor, Noise Model: Depolarizing, Fidelity: 5.086599304630501e-65, Error Rate: 1.0, Repetition: 10
Code: shor, Noise Model: Am

In [170]:
def evaluate_performance(backend, code_name):
    # Get noise models from your existing function
    noise_models = simulate_with_all_noise_models(0.05, backend)
    
    # Ensure noise_models is a list of dictionaries as returned by your simulate_with_all_noise_models function
    if not isinstance(noise_models, list):
        raise ValueError("The simulate_with_all_noise_models function should return a list.")

    if code_name == 'shor':
        qc_circuit = shor_code_circuit(include_measure=True)
    elif code_name == 'steane':
        qc_circuit = steane_code_circuit(include_measure=True)
    elif code_name == 'surface':
        qc_circuit = surface_code_circuit(include_measure=True)
    else:
        raise ValueError(f"Unknown code name: {code_name}")
    
    # Start memory and time profiling
    tracemalloc.start()
    start_time = time.time()
    
    qc_transpiled = transpile(qc_circuit, backend)
    
    performance_results = []
    
    for result in noise_models:
        if result['Code'] == code_name:
            noise_model_name = result['Noise Model']
            fidelity = result['Fidelity']
            error_rate = result['Error Rate']

            end_time = time.time()
            execution_time = end_time - start_time

            current, peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()

            print(f"Code: {code_name}, Noise Model: {noise_model_name}, Execution Time: {execution_time} seconds, Peak Memory Usage: {peak / 10**6} MB")
            
            performance_results.append({
                'Code': code_name,
                'Noise Model': noise_model_name,
                'Execution Time (Seconds)': execution_time,
                'Peak Memory Usage (MB)': peak / 10**6
            })
    
    return performance_results

# Performance analysis for Shor, Steane, and Surface codes on the current environment
for code_name in ['shor', 'steane', 'surface']:
    performance = evaluate_performance(backend, code_name)

    # Save performance results to a CSV file
    csv_filename = f'{code_name}_performance_comparison.csv'
    with open(csv_filename, 'w', newline='') as csvfile:
        fieldnames = ['Code', 'Noise Model', 'Execution Time (Seconds)', 'Peak Memory Usage (MB)']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for result in performance:
            writer.writerow(result)

    print(f"Results for {code_name} saved to {csv_filename}")


Code: shor, Noise Model: Depolarizing, Fidelity: 1.2390469098367252e-33, Error Rate: 0.125
Code: shor, Noise Model: Amplitude Damping, Fidelity: 1.2390469098367252e-33, Error Rate: 0.125
Code: shor, Noise Model: Phase Damping, Fidelity: 1.2390469098367252e-33, Error Rate: 0.125
Code: shor, Noise Model: Bit Flip, Fidelity: 1.2390469098367252e-33, Error Rate: 0.125
Code: shor, Noise Model: Phase Flip, Fidelity: 1.2390469098367252e-33, Error Rate: 0.125
Code: shor, Noise Model: Bit Phase Flip, Fidelity: 1.2390469098367252e-33, Error Rate: 0.125
Code: shor, Noise Model: Kraus, Fidelity: 2.331834530286036e-05, Error Rate: 0.125
Code: steane, Noise Model: Depolarizing, Fidelity: 0.06250000000000003, Error Rate: 0.125
Code: steane, Noise Model: Amplitude Damping, Fidelity: 0.06250000000000003, Error Rate: 0.125
Code: steane, Noise Model: Phase Damping, Fidelity: 0.06250000000000003, Error Rate: 0.125
Code: steane, Noise Model: Bit Flip, Fidelity: 0.06250000000000003, Error Rate: 0.125
Code: s

In [161]:
{
    "Depolarizing": create_depolarizing_noise_model,
    "Amplitude Damping": create_amplitude_damping_noise_model,
    # Add other noise models here
}


{'Depolarizing': <function __main__.create_depolarizing_noise_model(noise_level)>,
 'Amplitude Damping': <function __main__.create_amplitude_damping_noise_model(noise_level)>}

In [174]:
def analyze_resources(backend, code_name):
    if code_name == 'shor':
        qc_circuit = shor_code_circuit(include_measure=True)
    elif code_name == 'steane':
        qc_circuit = steane_code_circuit(include_measure=True)
    elif code_name == 'surface':
        qc_circuit = surface_code_circuit(include_measure=True)
    else:
        raise ValueError(f"Unknown code name: {code_name}")
    
    # Transpile the circuit for the backend
    qc_transpiled = transpile(qc_circuit, backend)
    
    # Calculate resources
    depth = qc_transpiled.depth()
    gate_counts = qc_transpiled.count_ops()
    qubit_count = qc_transpiled.num_qubits
    
    # Calculate qubit overhead (physical qubits needed minus logical qubit)
    logical_qubit_count = 1  # Assuming we're encoding one logical qubit in these codes
    qubit_overhead = qubit_count - logical_qubit_count
    
    print(f"Code: {code_name}")
    print(f"Circuit Depth: {depth}")
    print(f"Gate Counts: {gate_counts}")
    print(f"Total Qubit Count: {qubit_count}")
    print(f"Qubit Overhead: {qubit_overhead}")
    
    return {
        'Code': code_name,
        'Circuit Depth': depth,
        'Gate Counts': gate_counts,
        'Total Qubit Count': qubit_count,
        'Qubit Overhead': qubit_overhead
    }

# Analyze resources and qubit overhead for each code
for code_name in ['shor', 'steane', 'surface']:
    resources = analyze_resources(backend, code_name)
    
    # Save resource usage results to a CSV file
    csv_filename = f'{code_name}_resource_usage.csv'
    with open(csv_filename, 'w', newline='') as csvfile:
        fieldnames = ['Code', 'Circuit Depth', 'Gate Counts', 'Total Qubit Count', 'Qubit Overhead']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        writer.writerow(resources)

    print(f"Resource usage for {code_name} saved to {csv_filename}")


Code: shor
Circuit Depth: 10
Gate Counts: OrderedDict({'cx': 18, 'measure': 9, 'h': 7, 'save_statevector': 1, 'barrier': 1})
Total Qubit Count: 9
Qubit Overhead: 8
Resource usage for shor saved to shor_resource_usage.csv
Code: steane
Circuit Depth: 7
Gate Counts: OrderedDict({'measure': 7, 'cx': 6, 'h': 4, 'save_statevector': 1, 'barrier': 1})
Total Qubit Count: 7
Qubit Overhead: 6
Resource usage for steane saved to steane_resource_usage.csv
Code: surface
Circuit Depth: 14
Gate Counts: OrderedDict({'measure': 13, 'cx': 6, 'cz': 6, 'h': 3, 'save_statevector': 1, 'barrier': 1})
Total Qubit Count: 13
Qubit Overhead: 12
Resource usage for surface saved to surface_resource_usage.csv


In [185]:
!pip install pandas


  Using cached pandas-2.2.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp312-cp312-macosx_11_0_arm64.whl (11.3 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
